## **Webtoons - WebScraping Project**

<img src="https://media-exp1.licdn.com/dms/image/C561BAQF7mhzUFFpGQg/company-background_10000/0/1588199218989?e=2147483647&v=beta&t=jmdPaw4ITMyXHwfx5vv7QJ6TnzTNhYhpuX3wUfLrCgg" width="100%">

> **Disclaimer:** This is a personal project to practice webscraping skills and exploratory data analysis. I do not recommend to use for other purposes. Use it at your own risk.

### **Libraries**

All comics on webtoons are on only one page, but it has extra information on their unique page. The scraping process will use only the main tools. 

> **If you wanna replicate, maybe you need to install some of the packages with PIP command.**

In [1]:
import math
import requests
import pandas as pd

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

Let's define the URL to request the comics HTML page.

In [2]:
url = 'https://www.webtoons.com/en/genre'

Let's start the extracting process of the comics.

In [3]:
req = requests.get(url)
req.status_code

200

Great! Parsing the HTML to a BeautifulSoup object.

In [4]:
soup = BeautifulSoup(req.text, 'html.parser')

Each container has comics from a genre. It's a total of 16 containers and over 1400 comics. Let's collect all the comics.

In [5]:
containers = soup.findAll('ul', attrs={'class': 'card_lst'})
len(containers)

16

In [6]:
items = [i for c in containers for i in c.findAll('li')]
len(items)

1440

To collect the release date information requires knowing how many episodes pages have to make another request. Let's encapsulate it on a function. 

In [7]:
def get_item_released_information(url, page):
    req = requests.get(url, params={'page': page})
    if req.status_code != 200:
        raise requests.ConnectionError(f'failed to connect to {url}. [{req.status_code}]')
    
    bs = BeautifulSoup(req.text, 'html.parser')

    first_episode = bs.find('ul', attrs={'id': '_listUl'}).findAll('li')[-1]
    released_data = first_episode.find('span', attrs={'class': 'date'}).text.strip()
    
    return released_data

The comic information is on a unique page. Let's encapsulate it too on a function.

In [8]:
def get_item_information(url):
    req = requests.get(url)
    if req.status_code != 200:
        raise requests.ConnectionError(f'failed to connect to {url}. [{req.status_code}]')
    
    bs = BeautifulSoup(req.text, 'html.parser')
    
    item_data = {}
    
    item_data['webtoon_id'] = url.split('=')[-1]
    item_data['title'] = bs.find('h1', attrs={'class': 'subj'}).text.strip() 
    item_data['genre'] = bs.find('h2', attrs={'class': 'genre'}).text.strip()
    item_data['thumbnail'] = bs.find('span', attrs={'class': 'thmb'}).img['src']
    item_data['summary'] = bs.find('p', attrs={'class': 'summary'}).text.strip()
    item_data['episodes'] = bs.find('ul', attrs={'id': '_listUl'}).findAll('li')[0]['data-episode-no']
    
    authors_container = bs.find('div', attrs={'class': '_authorInnerContent'})
    authors_categories = authors_container.findAll('p', attrs={'class': 'by'})
    authors_names = authors_container.findAll('h3', attrs={'class': 'title'})
    for c, n in zip(authors_categories, authors_names):
        item_data[c.text.strip()] = n.text.strip()
        
    stats_container = bs.find('p', attrs={'class': 'summary'}).parent.find('ul').findAll('li')
    for s in stats_container:
        item_data[s.span.text.strip()] = s.em.text.strip()
    
    last_page = int(math.ceil(int(item_data['episodes']) / 10))
    item_data['released_date'] = get_item_released_information(url, last_page)
    item_data['url'] = url
    
    return item_data

The collected data will be on a list.

In [18]:
data = []

Now, let's start the looping process to collect all the information!

In [19]:
done = []

In [20]:
for i in tqdm(items):    
    href = i.a['href']
    if href in done:
        continue
        
    item_data = get_item_information(href)
    item_data['cover'] = i.find('img')['src']
    item_data['likes'] = i.find('em', attrs={'class': 'grade_num'}).text.strip()  
    
    data.append(item_data)
    done.append(href)

  0%|          | 0/1440 [00:00<?, ?it/s]

The scraping process took around 35 minutes, with two requests per comic. 

As the data are a list of dictionaries, we don't need to define all the column's names. Let's parse the data to a DataFrame.

In [21]:
df = pd.DataFrame(data)
df.head()

,webtoon_id,title,genre,thumbnail,summary,episodes,Created by,view,subscribe,grade,released_date,url,cover,likes,Written by,Art by,Adapted by,Original work by,Assisted by
0,1218,Let's Play,Romance,https://webtoon-phinf.pstatic.net/20210629_103...,"She’s young, single and about to achieve her d...",171,Leeanne M. Krecic (Mongie),606.5M,4.6M,9.59,"Nov 6, 2017",https://www.webtoons.com/en/romance/letsplay/l...,https://webtoon-phinf.pstatic.net/20210629_163...,37.2M,NaN,NaN,NaN,NaN,NaN
1,1436,True Beauty,Romance,https://webtoon-phinf.pstatic.net/20210129_175...,"After binge-watching beauty videos online, a s...",197,Yaongyi,874M,7M,9.53,"Aug 15, 2018",https://www.webtoons.com/en/romance/truebeauty...,https://webtoon-phinf.pstatic.net/20210129_65/...,46.2M,NaN,NaN,NaN,NaN,NaN
2,2135,The Remarried Empress,Fantasy,https://webtoon-phinf.pstatic.net/20200904_29/...,Navier Ellie Trovi was an empress perfect in e...,110,NaN,231.3M,2.5M,9.87,"Sep 5, 2020",https://www.webtoons.com/en/fantasy/the-remarr...,https://webtoon-phinf.pstatic.net/20200904_268...,21.2M,Alphatart,Sumpul,HereLee,NaN,NaN
3,1798,Midnight Poppy Land,Romance,https://webtoon-phinf.pstatic.net/20191119_132...,After making a grisly discovery in the country...,99,Lilydusk,198.8M,2.3M,9.80,"Nov 22, 2019",https://www.webtoons.com/en/romance/midnight-p...,https://webtoon-phinf.pstatic.net/20191119_163...,13.5M,NaN,NaN,NaN,NaN,NaN
4,3416,Reunion,Romance,https://webtoon-phinf.pstatic.net/20220311_196...,"After moving away for a decade, Rhea returns t...",9,stephattyy,7.1M,"629,872",9.77,"Mar 17, 2022",https://www.webtoons.com/en/romance/reunion/li...,https://webtoon-phinf.pstatic.net/20220311_14/...,"570,151",NaN,NaN,NaN,NaN,NaN


And finally, let's save the data.

In [24]:
df.to_csv('data.csv', index=False)